<a href="https://colab.research.google.com/github/cwilbar04/nba-predictions/blob/main/notebooks/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.colab import files

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [8]:
my_project_id = "nba-predictions-dev"
my_project_id

'nba-predictions-dev'

In [11]:
%%bigquery --project nba-predictions-dev
  SELECT max(game_date) from `nba.raw_basketballreference_game`


ERROR:
 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/nba-predictions-dev/jobs: Access Denied: Project project_id: "nba-predictions-dev"
gaia_id: 188994400757
: User does not have bigquery.jobs.create permission in project nba-predictions-dev.

(job ID: 2fce96d7-a264-490b-8f4e-31f631d18b28)

                 -----Query Job SQL Follows-----                  

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT max(game_date) from `nba.raw_basketballreference_game`
    |    .    |    .    |    .    |    .    |    .    |    .    |
